In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup
from datetime import datetime

In [2]:
base_url = "https://premierleague.com/match/"

In [3]:
first_match_id = "5567" # should be 5567 for start of 06/07 season
example_url = base_url + first_match_id
response = requests.get(example_url,headers={'User-Agent': 'Custom'})
page = soup(response.content, "html.parser")

In [7]:
example_events = get_event_data(page)
example_events

[]

In [55]:
def get_match_info(page):
    top_bar_match_info = page.findAll("div",{"class":"matchInfo"})[0]
    date_seconds = top_bar_match_info.find("div",{"class":"matchDate renderMatchDateContainer"}).attrs["data_kickoff"]
    location_info = top_bar_match_info.find("div",{"class":"stadium"}).text.strip().split(",")
    teams_data = page.findAll("span",{"class":"long"})
    
    
    date = datetime.fromtimestamp(int(seconds)/1000).strftime("%d-%m-%Y")
    referee = top_bar_match_info.find("div",{"class":"referee"}).text.strip()
    stadium = location_info[0].strip()
    location = location_info[1].strip()
    attendance = top_bar_match_info.find("div",{"class":"attendance hide-m"}).text.replace("Att: ","").replace(",","")
    home_team = teams_data[0].text
    away_team = teams_data[1].text
    score = page.findAll("div",{"class":"score fullTime"})[0].text

In [6]:
def get_event_data(page):
    home_events = page.findAll("div",{"class":"event home"})
    away_events = page.findAll("div",{"class":"event away"})
    output_events = []
    for event in home_events:
        type_text = event.findAll("span",{"class":"visuallyHidden"})[0].text
        # Find type of event
        if type_text == "Substitution":
            mins = event.find("time",{"class":"min"}).text[:2]
            names = event.findAll("a",{"class":"name"})
            player_off = names[0].text.split(".")[1].split("\n")[0].strip()
            player_on = names[1].text.split(".")[1].split("\n")[0].strip()
            sub = mins + ":" + player_off + "," + player_on
            output_events.append({"home-sub":sub})
        elif type_text == "Yellow Card":
            mins = event.find("time",{"class":"min"}).text[:2]
            name = event.find("a",{"class":"name"}).text.split(".")[1].strip()
            card = mins + ":" + name
            output_events.append({"home-yellow":card})
        elif type_text == "Red Card":
            mins = event.find("time",{"class":"min"}).text[:2]
            name = event.find("a",{"class":"name"}).text.split(".")[1].strip()
            card = mins + ":" + name
            output_events.append({"home-red":card})
        elif type_text == "Goal":
            mins = event.find("time",{"class":"min"}).text[:2]
            name = event.find("a",{"class":"name"}).text.split(".")[1].strip()
            goal = mins + ":" + name
            output_events.append({"home-goal":goal})
    # Repeat same steps for away_events
    for event in away_events:
        type_text = event.findAll("span",{"class":"visuallyHidden"})[0].text
        # Find type of event
        if type_text == "Substitution":
            mins = event.find("time",{"class":"min"}).text[:2]
            names = event.findAll("a",{"class":"name"})
            player_off = names[0].text.split(".")[1].split("\n")[0].strip()
            player_on = names[1].text.split(".")[1].split("\n")[0].strip()
            sub = mins + ":" + player_off + "," + player_on
            output_events.append({"away-sub":sub})
        elif type_text == "Yellow Card":
            mins = event.find("time",{"class":"min"}).text[:2]
            name = event.find("a",{"class":"name"}).text.split(".")[1].strip()
            card = mins + ":" + name
            output_events.append({"away-yellow":card})
        elif type_text == "Red Card":
            mins = event.find("time",{"class":"min"}).text[:2]
            name = event.find("a",{"class":"name"}).text.split(".")[1].strip()
            card = mins + ":" + name
            output_events.append({"away-red":card})
        elif type_text == "Goal":
            mins = event.find("time",{"class":"min"}).text[:2]
            name = event.find("a",{"class":"name"}).text.split(".")[1].strip()
            goal = mins + ":" + name 
            output_events.append({"away-goal":goal})
    return output_events

In [ ]:
def event_data_to_features(event_data):
    # Outputs are: home_sub1, home_sub2, home_sub3, home_yellows, home_reds, home_goalscorers
    # Then same for away
    home_sub1, home_sub2, home_sub3, home_yellows, home_reds, home_goalscorers = ""*6
    away_sub1, away_sub2, away_sub3, away_yellows, away_reds, away_goalscorers = ""*6
    for d in output_events:
    